<a href="https://colab.research.google.com/github/awaiskhan005/DATA-SCIENCE-AND-AI-/blob/main/Bertt%20MILLIE%20PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 17.3 MB/s eta 0:00:00


In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 23.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
!pip install transformers

In [ ]:
!apt-get install -y google-drive-ocamlfuse

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package google-drive-ocamlfuse


# Loading Articles

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import urllib
titles = []
articles = []
# Specify the path to the articles directory on your Google Drive
data_dir = '/content/drive/My Drive/articles'

# Read in the article titles and articles
for filename in os.listdir(data_dir):
    # Open the file
    with open(os.path.join(data_dir, filename), "rb") as f:
        # Decode any characters that aren't allowed in URLs.
        # e.g., Lord_of_Storm%27s_End --> Lord_of_Storm's_End
        title = urllib.parse.unquote(filename[:-4])

        # Replace underscores with spaces.
        title = title.replace('_', ' ')

        # Oddly, there is one file that's just named ".txt". Bet it was the
        # base URL of the website.
        if (len(title) == 0) or (len(title.strip()) == 0):
            print('Empty title for', filename)
            continue

        # Add it to the list.
        titles.append(title)

        # Read in the article text.
        articles.append(f.read().decode('latin-1'))

# Print the number of articles
print('There are {:,} articles.'.format(len(articles)))

There are 6 articles.


In [ ]:
titles[0:6]

['Hair growth and rejuvenation',
 'Afro-textured hair',
 'Hair and Food',
 'Diet and hair loss effects of nutrient deficiency and supple',
 'Diversity in human hair growth',
 'Hair Question and Answer Pairs']

In [ ]:
articles[0]

'= Hair growth and rejuvenation = \r\r== Abstract == \rHair has psychological and sociological importance throughout the ages in framing the personality and general appearance of an individual. Significant progress is being made on discovering an effective and safe drug for hair growth. Angiogenesis, androgen antagonism, vasodilation, potassium channel opening, and 5-alpha reductase inhibition are the major non-surgical therapeutic strategies of hair growth promotion. In spite of a flood of drugs claiming to be useful as hair growth promoters, more rational strategies, which can target the problem areas or stages of the hair growth cycle effectively, are still awaited. This article highlights the developments in hair rejuvenation strategies and reviews the potential of herbal drugs as safer and effective alternatives.\r\r== Introduction == \rHair has been a sign of beauty and a contribution to an individualÕs personality since time immemorial. Alopecia (baldness), a dermatological diso

# Chunking the articles

In [ ]:
print('Before splitting, {:,} articles.\n'.format(len(titles)))

# We have two lists, 'titles' and 'articles'.

passage_titles = []
passages = []

print('Splitting...')

# For each article and its title...
for i in range(0, len(articles)):

    title = titles[i]
    article = articles[i]

    # Skip over any without contents.
    if len(article) == 0:
        print('Skipping empty article:', title)
        continue

    # Split the text on whitespace.
    # By default, this removes all whitespace, including newline and tab
    # characters.
    words = article.split()

    # Loop over the words, incrementing by 100.
    for i in range(0, len(words), 100):

        # Select the next 100 words.
        # Python slices automatically stop at the end of the array.
        chunk_words = words[i : i + 100]

        # Recombine the words into a passage by joining with whitespace.
        chunk = " ".join(chunk_words)

        # Remove any trailing whitespace.
        chunk = chunk.strip()

        # To avoid a possible edge case, skip any empty chunks.
        if len(chunk) == 0:
            continue

        # Store the chunk. Every chunk in the article uses the article title.
        passage_titles.append(title)
        passages.append(chunk)

print('  Done.\n')

chunked_corpus = {'title': passage_titles, 'text': passages}

print('After splitting, {:,} "passages".'.format(len(chunked_corpus['title'])))

Before splitting, 6 articles.

Splitting...
  Done.

After splitting, 187 "passages".


# DPR Embeddings

In [ ]:
from transformers import DPRContextEncoderTokenizerFast

# Load the tokenizer.
ctx_tokenizer = DPRContextEncoderTokenizerFast.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/492 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


In [ ]:
num_passages = len(chunked_corpus['title'])

print('Tokenizing {:,} passages for DPR...'.format(num_passages))

# Tokenize the whole dataset! This will take ~15 to 20 seconds.
outputs = ctx_tokenizer(
    chunked_corpus["title"],
    chunked_corpus["text"],
    truncation=True,
    padding="longest",
    return_tensors="pt",
)

print('  DONE.')

# `input_ids` holds the encoded tokens for the entire corpus.
input_ids = outputs["input_ids"]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Tokenizing 187 passages for DPR...
  DONE.


In [ ]:
print(input_ids.shape)

torch.Size([187, 261])


# "Encoding" Generating Embeddings


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available!')
    # You could use the CPU, but I don't recommend it!
    #device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
from transformers import DPRContextEncoder

# "The DPR context encoder model to use.
# Either 'facebook/dpr-ctx_encoder-single-nq-base' or
#        'facebook/dpr-ctx_encoder-multiset-base'"
#
ctx_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-multiset-base")

# Move the encoder model to the GPU.
ctx_encoder = ctx_encoder.to(device=device)

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-multiset-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import time
import math

# We're running the model forward only, so no need for gradients.
torch.set_grad_enabled(False)

# Track elapsed time for progress updates.
t0 = time.time()

# Track the current batch number, also for progress updates.
step = 0

# How many passages to process per batch.
batch_size = 16

# Get the number of passages in the dataset, we'll use this in a few places.
num_passages = input_ids.size()[0]

# Calculate the number of batches in the dataset.
num_batches = math.ceil(num_passages / batch_size)

# As we embed the passages in batches, accumulate them in this list.
embeds_batches = []

print('Generating embeddings for {:,} passages...'.format(num_passages))

# For each batch of passages...
for i in range(0, num_passages, batch_size):

    # Progress update every 100 batches.
    if step % 100 == 0 and not step == 0:
        # Calculate elapsed time in minutes.
        elapsed = format_time(time.time() - t0)

        # Report progress.
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, num_batches, elapsed))

    # Select the next batch.
    batch_ids = input_ids[i:i + 16, :]

    # Move them to the GPU.
    batch_ids = batch_ids.to(device)

    # Run the encoder!
    outputs = ctx_encoder(
        batch_ids,
        return_dict=True
    )

    # The embeddings are in the field "pooler_output"
    embeddings = outputs["pooler_output"]

    # Bring the embeddings back over from the GPU and convert to numpy (out of
    # pytorch)
    embeddings = embeddings.detach().cpu().numpy()

    embeds_batches.append(embeddings)

    step += 1

print('  DONE.')

Generating embeddings for 187 passages...
  DONE.


In [ ]:
import numpy as np

# Combine the results across all batches.
embeddings = np.concatenate(embeds_batches, axis=0)

print('Size of dataset embeddings:', embeddings.shape)

Size of dataset embeddings: (187, 768)


In [ ]:
# Import the FAISS library for indexing the corpus.
import faiss

# "The dimension of the embeddings to pass to the HNSW Faiss index."
dim = 768

# "The number of bi-directional links created for every new element during the
# HNSW index construction."
m = 128

# Let's use the Faiss implementation of HNSW for fast approximate nearest neighbor search
index = faiss.IndexHNSWFlat(dim, m, faiss.METRIC_INNER_PRODUCT)

In [ ]:
print('Building the FAISS index...')

# Track elapsed time for progress updates.
t0 = time.time()

index.train(embeddings)

index.add(embeddings)

print('  DONE.')

print('  Adding embeddings to index took', format_time(time.time() - t0))


Building the FAISS index...
  DONE.
  Adding embeddings to index took 0:00:00


In [ ]:
from transformers import DPRQuestionEncoder

# "The DPR context encoder model to use.
# Either 'facebook/dpr-ctx_encoder-single-nq-base' or
#        'facebook/dpr-ctx_encoder-multiset-base'"

q_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-multiset-base")

# Move the encoder model to the GPU.
q_encoder = q_encoder.to(device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-question_encoder-multiset-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from transformers import DPRQuestionEncoderTokenizerFast

# Load the tokenizer.
q_tokenizer = DPRQuestionEncoderTokenizerFast.from_pretrained("facebook/dpr-question_encoder-multiset-base")


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

# Questions

In [ ]:
# Tokenize the question.
input_ids = q_tokenizer.encode("What are the phases of the hair growth cycle?", return_tensors="pt")

# Move the question over to the GPU.
input_ids = input_ids.to(device)

# Run the question through BERT and generate the question embedding.
outputs = q_encoder(input_ids)

# The embedding is stored in the `pooler_output` property.
q_embed = outputs['pooler_output']

# Our FAISS index is on the CPU, not the GPU, so we need to transfer this
# question embedding over to the CPU to do our search.
q_embed = q_embed.cpu().numpy()

# Check out the embedding's size, out of curiosity :)
print("Query embedding:", q_embed.shape)

Query embedding: (1, 768)


In [ ]:
# Find the k=3 most similar passages to the question embedding `q_embed`.
D, I = index.search(q_embed, k=3)

# Print out the indeces and simlarity scores--we'll print out the passage text
# next.
print('Closest matching indeces:', I)
print('Inner Products:', D)

Closest matching indeces: [[6 9 5]]
Inner Products: [[83.79413 83.3949  80.89393]]


In [ ]:
import textwrap

# Wrap text to 80 characters.
wrapper = textwrap.TextWrapper(width=80)

# For each of the top 'k' results..
for i in I[0]:

    print('Index:', i)

    # Retrieve passage and its title.
    title = chunked_corpus['title'][i]
    passage = chunked_corpus['text'][i]

    print('Article Title:   ', title, '\n')

    print('Passage:')
    print(wrapper.fill(passage))

    print('')


Index: 6
Article Title:    Hair growth and rejuvenation 

Passage:
involvement of cell differentiation, epithelialÐ mesenchymal interactions, stem
cell augmentation, pattern formation, apoptosis (programmed cell death), cell
and organ growth cycles, and pigmentation. The most important reason for
studying the cycling of the hair follicle is that the follicle is a regenerating
system. By traversing the phases of the cycle (growth, regression, resting,
shedding, then growth again), the follicle demonstrates the unusual ability to
completely regenerate itself. Normal hair follicles cycle between a growth stage
(anagen), a degenerative stage (catagen), a resting stage (telogen) and a
shedding stage (exogen). The scalp hairs have a relatively long-life cycle: the
anagen stage

Index: 9
Article Title:    Hair growth and rejuvenation 

Passage:
of the primitive epithelium to form a placode. Below the placode dermal cells
aggregate and thereafter the epithelium grows down as a finger to produc

In [ ]:
chunked_corpus = {'title': passage_titles, 'text': passages}

In [ ]:
from datasets import Dataset
import pandas as pd

# Create a DataFrame from the dictionary.
df = pd.DataFrame(chunked_corpus)

# Convert the DataFrame into a huggingface Dataset object.
dataset = Dataset.from_pandas(df)

# Check out the object.
print(dataset)

Dataset({
    features: ['title', 'text'],
    num_rows: 187
})


In [ ]:
embs = []

# For each embedding...
for i in range(embeddings.shape[0]):

    # Add it to the list
    embs.append(embeddings[i, :])

In [ ]:
# Add the embeddings to the dataset as a new column.
# This is not done in place, so make sure to capture the return value.
dataset = dataset.add_column("embeddings", embs)

# Display the dataset contents.
dataset

Dataset({
    features: ['title', 'text', 'embeddings'],
    num_rows: 187
})

In [ ]:
# Let's use the Faiss implementation of HNSW for fast approximate nearest neighbor search
index = faiss.IndexHNSWFlat(dim, m, faiss.METRIC_INNER_PRODUCT)

dataset.add_faiss_index(column="embeddings", index_name="embeddings", custom_index=index, faiss_verbose=True)

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset({
    features: ['title', 'text', 'embeddings'],
    num_rows: 187
})

# Retriever


In [ ]:
from transformers import RagRetriever


retriever = RagRetriever.from_pretrained(

    "facebook/rag-sequence-nq", # The specific pre-trained model we'll use.

    use_dummy_dataset=False, # The dummy dataset is reduced-size Wikipedia,
                             # I think they included this for demonstration?

    indexed_dataset=dataset, # Pass in our own GoT dataset!

    index_name="embeddings", # Specify the name of the FAISS index we created
                             # in section 5.1.1.

)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

(…)_encoder_tokenizer/tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

question_encoder_tokenizer/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)ncoder_tokenizer/special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

(…)enerator_tokenizer/tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

generator_tokenizer/vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

generator_tokenizer/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)erator_tokenizer/special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizerFast'.


In [ ]:
from transformers import RagTokenizer

tokenizer = RagTokenizer.from_pretrained(
    "facebook/rag-sequence-nq"
)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

In [ ]:
from transformers import RagSequenceForGeneration

model = RagSequenceForGeneration.from_pretrained(
    "facebook/rag-sequence-nq",
    retriever=retriever
)


pytorch_model.bin:   0%|          | 0.00/2.06G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/rag-sequence-nq were not used when initializing RagSequenceForGeneration: ['rag.question_encoder.question_encoder.bert_model.pooler.dense.bias', 'rag.question_encoder.question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing RagSequenceForGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RagSequenceForGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Example Question

In [ ]:
import time

t0 = time.time()

question = "What are the phases of the hair growth cycle?"

# Tokenize the question.
input_ids = tokenizer.question_encoder(question, return_tensors="pt")["input_ids"]

# Give the question to RAG and have it generate an answer!
generated = model.generate(input_ids)

# Convert the answer tokens back into a single string.
generated_string = tokenizer.batch_decode(generated, skip_special_tokens=True)[0]

print ("Q: " + question)
print ("A: " + generated_string)

print('\nResponse took %.2f seconds' % (time.time() - t0))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Q: What are the phases of the hair growth cycle?
A:  anagen

Response took 162.36 seconds


### The answer is incomplete.

correct answer is - The phases of the hair growth cycle are anagen (growth), catagen (regression),
telogen (resting), and exogen (shedding).

In [ ]:
def ask_question(question):
    t0 = time.time()

    input_ids = tokenizer.question_encoder(question, return_tensors="pt")["input_ids"]
    generated = model.generate(input_ids)
    generated_string = tokenizer.batch_decode(generated, skip_special_tokens=True)[0]

    print ("Q: " + question)
    print ("A: '{:}'".format(generated_string))

    print('\nResponse took %.2f seconds' % (time.time() - t0))

In [ ]:
ask_question('What are the different types of alopecia?')

Q: What are the different types of alopecia?
A: ' alopecia totalis'

Response took 179.42 seconds


### It only gave me one type of alopecia
 correct answer is - The different types of alopecia include male pattern baldness, female pattern
baldness, alopecia areata, alopecia totalis, and alopecia universalis.

In [ ]:
ask_question("What is considered a normal amount of daily hair loss?")

Q: What is considered a normal amount of daily hair loss?
A: ' 2.5'

Response took 226.35 seconds


### This answer is wrong.

correct answer is - On average, shedding 50–125 hairs per day is considered normal, depending on sex,
but a loss of more than 100 hairs per day is considered pathological.